In [19]:
import cv2
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import pandas as pd
import os
import shutil

In [20]:
def average_BGR(image):
    avg_blue = image[:,:,0].mean()
    avg_green = image[:,:,1].mean()
    avg_red = image[:,:,2].mean()

    return [avg_blue,avg_green,avg_red]

def average_HSV(image):
    image = cv2.cvtColor(image,cv2.COLOR_BGR2HSV)
    avg_hue = image[:,:,0].mean()
    avg_saturation = image[:,:,1].mean()
    avg_value = image[:,:,2].mean()

    return avg_hue, avg_saturation, avg_value

In [21]:
list_avg_blue = []
list_avg_green = []
list_avg_red = []

list_avg_hue=[]
list_avg_saturation=[]
list_avg_value=[]
X1 = []
X2 = []
list_image = []
list_image_path=[]
for dirname, _, filenames in os.walk('D:\PTIT\CSDLDPT\Multimedia-Database-System\Mix'):
    for filename in filenames:
        image_path = os.path.join(dirname, filename)
        list_image_path.append(image_path)
        image = cv2.imread(image_path)
        list_image.append(image)
        avg_BGR = average_BGR(image)
        X1.append(avg_BGR)
        list_avg_blue.append(avg_BGR[0])
        list_avg_green.append(avg_BGR[1])
        list_avg_red.append(avg_BGR[2])
        avg_HSV= average_HSV(image)
        list_avg_hue.append(avg_HSV[0])
        list_avg_saturation.append(avg_HSV[1])
        list_avg_value.append(avg_HSV[2])
        X2.append(avg_HSV)

In [23]:
from sklearn.cluster import KMeans

cluster = 11

kmeans = KMeans(n_clusters=cluster,random_state =0)

X2 = np.array(X2)

kmeans.fit(X2)


print('Centers found by scikit-learn:')
print(kmeans.cluster_centers_)

pred_label = kmeans.predict(X2)


output_dir = 'D:\PTIT\CSDLDPT\Multimedia-Database-System\kmeans1'

# Kiểm tra xem thư mục tồn tại chưa, nếu không, tạo mới
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Tạo đường dẫn đến file
output_file = os.path.join(output_dir, 'cluster_labels.txt')


# Mở file để ghi dữ liệu
with open(output_file, 'w') as file:
    # Ghi nhãn của từng điểm dữ liệu
    np.savetxt(file, kmeans.labels_, fmt='%d')


output_file = os.path.join(output_dir, 'cluster_center.txt')

with open(output_file, 'w') as file:
    # Ghi trung tâm của các cụm

    np.savetxt(file, kmeans.cluster_centers_)


dict = {}

for i in range(len(list_image_path)):
    dict[list_image_path[i]] = pred_label[i]

for i in range(cluster):
    folder_name = str(i)
    os.mkdir(os.path.join('D:\PTIT\CSDLDPT\Multimedia-Database-System\kmeans1', folder_name))

for key, value in dict.items():
    for dirname, child_folders, filenames in os.walk('kmeans1'):
        for folder in child_folders:
            if str(value) == folder:
                shutil.copy(key, os.path.join(dirname, folder))


Centers found by scikit-learn:
[[ 74.22933333  15.32813333 165.43575333]
 [ 22.57452     92.76961333 173.80278667]
 [ 94.85196     94.31786    107.31125333]
 [ 17.480295    53.495255   222.478025  ]
 [ 39.788485   127.585285   100.630485  ]
 [ 46.77166562  55.72717812 155.56149063]
 [ 20.26515333 172.23953333 173.90971333]
 [ 24.29108462 105.83596923 200.83433846]
 [ 27.44935714  26.31848571 179.77884286]
 [ 21.954924    69.96274    197.133108  ]
 [ 22.03905926  69.8302037  144.14539259]]


C:\Users\ADMIN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [36]:
def kmeans_2(folder_path, mod):
    images = []
    list_image_path = []
    for filename in os.listdir(folder_path):
        # print(folder_path)
        image_path = os.path.join(folder_path, filename)
        # print(image_path)
        list_image_path.append(image_path)
        image = cv2.imread(image_path)
        # print(image)
        avg_HSV= average_HSV(image)
        images.append(avg_HSV)
    
    kmeans_second = KMeans(n_clusters=mod,random_state =0)
    kmeans_second.fit(np.array(images))
    label = kmeans_second.predict(images)
    
    dict = {}
    for i in range((len(list_image_path))):
        dict[list_image_path[i]] = label[i]
    
    for key, value in dict.items():
        print(key, ':', value)

    for i in range(mod):
        folder_name = str(i)
        os.mkdir(os.path.join(folder_path, folder_name))

    for key, value in dict.items():
        for dirname, child_folders, _ in os.walk(folder_path):
            for folder in child_folders:
                if str(value) == folder:
                    shutil.move(key, os.path.join(dirname, folder))

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Tạo đường dẫn đến file
    output_file = os.path.join(folder_path, 'cluster_labels.txt')


    # Mở file để ghi dữ liệu
    with open(output_file, 'w') as file:
        # Ghi nhãn của từng điểm dữ liệu
        np.savetxt(file, kmeans.labels_, fmt='%d')


    output_file = os.path.join(output_dir, 'cluster_center.txt')

    with open(output_file, 'w') as file:
        # Ghi trung tâm của các cụm

        np.savetxt(file, kmeans.cluster_centers_)    

folder = 'D:\PTIT\CSDLDPT\Multimedia-Database-System\kmeans1'

dict_km2 = {}

for i in range(cluster):
    folder_path = os.path.join(folder, str(i))
    mod = len(os.listdir(folder_path)) // 13
    if mod > 1:
        kmeans_2(folder_path, mod)
        

# print(result)

C:\Users\ADMIN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\ADMIN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\ADMIN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set

In [ ]:
all_list_image=[]
for i in range(len(pred_label)):
  all_list_image.append((pred_label[i],None,list_image_path[i],X1[i],X2[i]))
  if pred_label[i]==10:
    print(list_image_path[i])

D:\PTIT\CSDLDPT\Multimedia-Database-System\Mix\Avocado (01).jpg
D:\PTIT\CSDLDPT\Multimedia-Database-System\Mix\Avocado (02).jpg
D:\PTIT\CSDLDPT\Multimedia-Database-System\Mix\Avocado (03).jpg
D:\PTIT\CSDLDPT\Multimedia-Database-System\Mix\Avocado (04).jpg
D:\PTIT\CSDLDPT\Multimedia-Database-System\Mix\Avocado (05).jpg
D:\PTIT\CSDLDPT\Multimedia-Database-System\Mix\Avocado (06).jpg
D:\PTIT\CSDLDPT\Multimedia-Database-System\Mix\Avocado (10).jpg
D:\PTIT\CSDLDPT\Multimedia-Database-System\Mix\Avocado (11).jpg
D:\PTIT\CSDLDPT\Multimedia-Database-System\Mix\Avocado (13).jpg
D:\PTIT\CSDLDPT\Multimedia-Database-System\Mix\Avocado (15).jpg
D:\PTIT\CSDLDPT\Multimedia-Database-System\Mix\Pineapple (01).jpg
D:\PTIT\CSDLDPT\Multimedia-Database-System\Mix\Pineapple (02).jpg
D:\PTIT\CSDLDPT\Multimedia-Database-System\Mix\Pineapple (03).jpg
D:\PTIT\CSDLDPT\Multimedia-Database-System\Mix\Pineapple (04).jpg
D:\PTIT\CSDLDPT\Multimedia-Database-System\Mix\Pineapple (05).jpg
D:\PTIT\CSDLDPT\Multimedia-Dat

In [ ]:
all_list_array=[]
for i in range(len(pred_label)):
   all_list_array.append((pred_label[i],X1[i],X2[i],list_image_path[i]))
print(all_list_array)

[(10, [136.8012, 154.0514, 151.8798], array([ 20.8068,  80.2229, 154.0876]), 'D:\\PTIT\\CSDLDPT\\Multimedia-Database-System\\Mix\\Avocado (01).jpg'), (10, [131.1489, 147.4319, 145.4068], array([ 21.2552,  84.1484, 147.4488]), 'D:\\PTIT\\CSDLDPT\\Multimedia-Database-System\\Mix\\Avocado (02).jpg'), (10, [130.5005, 142.9855, 140.8643], array([ 21.2388,  79.8446, 143.0017]), 'D:\\PTIT\\CSDLDPT\\Multimedia-Database-System\\Mix\\Avocado (03).jpg'), (10, [132.097, 143.1298, 140.8104], array([ 21.7633,  77.0261, 143.1767]), 'D:\\PTIT\\CSDLDPT\\Multimedia-Database-System\\Mix\\Avocado (04).jpg'), (10, [138.8499, 150.6375, 148.069], array([ 22.6615,  72.3086, 150.7136]), 'D:\\PTIT\\CSDLDPT\\Multimedia-Database-System\\Mix\\Avocado (05).jpg'), (10, [142.5988, 158.1907, 155.8741], array([ 21.9865,  73.9986, 158.2611]), 'D:\\PTIT\\CSDLDPT\\Multimedia-Database-System\\Mix\\Avocado (06).jpg'), (4, [87.0346, 102.1, 100.3139], array([ 28.4356, 107.7157, 102.1632]), 'D:\\PTIT\\CSDLDPT\\Multimedia-Datab

In [ ]:
result = []
result_pred_label2 = []
for i in range(11):
  len_label = len([key for key,value1,value2,image in all_list_array if key == i])
  count = len_label//13
  if count >1:
    kmeans2_label1 = KMeans(n_clusters=count ,random_state =0)

    result = [value2 for key,value1,value2,image in all_list_array if key == i]
    result = np.array(result)

    kmeans2_label1.fit(result)

    print('Centers found by scikit-learn:')
    print(kmeans.cluster_centers_)
    result_pred_label2 = (kmeans2_label1.predict(result))
    # kmeans_display(result, result_pred_label2)
    for index, (key1, key2, image, value1,value2) in enumerate(all_list_image):
      for i in range(len(result_pred_label2)):
          if (value2 == result[i]).all():
              # Tạo một bản sao của tuple hiện tại
              updated_tuple = list(all_list_image[index])
              # Thay đổi giá trị của key2 trong danh sách
              updated_tuple[1] = result_pred_label2[i]
              # Chuyển đổi danh sách đã được cập nhật thành tuple và gán lại vào all_list_image
              all_list_image[index] = tuple(updated_tuple)

C:\Users\ADMIN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Centers found by scikit-learn:
[[ 74.22933333  15.32813333 165.43575333]
 [ 22.57452     92.76961333 173.80278667]
 [ 94.85196     94.31786    107.31125333]
 [ 17.480295    53.495255   222.478025  ]
 [ 39.788485   127.585285   100.630485  ]
 [ 46.77166562  55.72717812 155.56149063]
 [ 20.26515333 172.23953333 173.90971333]
 [ 24.29108462 105.83596923 200.83433846]
 [ 27.44935714  26.31848571 179.77884286]
 [ 21.954924    69.96274    197.133108  ]
 [ 22.03905926  69.8302037  144.14539259]]
Centers found by scikit-learn:

C:\Users\ADMIN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\ADMIN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)



[[ 74.22933333  15.32813333 165.43575333]
 [ 22.57452     92.76961333 173.80278667]
 [ 94.85196     94.31786    107.31125333]
 [ 17.480295    53.495255   222.478025  ]
 [ 39.788485   127.585285   100.630485  ]
 [ 46.77166562  55.72717812 155.56149063]
 [ 20.26515333 172.23953333 173.90971333]
 [ 24.29108462 105.83596923 200.83433846]
 [ 27.44935714  26.31848571 179.77884286]
 [ 21.954924    69.96274    197.133108  ]
 [ 22.03905926  69.8302037  144.14539259]]
Centers found by scikit-learn:
[[ 74.22933333  15.32813333 165.43575333]
 [ 22.57452     92.76961333 173.80278667]
 [ 94.85196     94.31786    107.31125333]
 [ 17.480295    53.495255   222.478025  ]
 [ 39.788485   127.585285   100.630485  ]
 [ 46.77166562  55.72717812 155.56149063]
 [ 20.26515333 172.23953333 173.90971333]
 [ 24.29108462 105.83596923 200.83433846]
 [ 27.44935714  26.31848571 179.77884286]
 [ 21.954924    69.96274    197.133108  ]
 [ 22.03905926  69.8302037  144.14539259]]


In [ ]:
tmp = [[] for _ in range(11)]
for key1, key2, image, value1, value2 in all_list_image:
  if key2 != None:
    tmp[key1].append([key2,image,value1,value2])

for index, (i) in enumerate(tmp):
  if len(i)>0:
    print(index)
    for key2,image,value1,value2 in i:
      if key2==0:
        print(image)
    print()
    for key2,image,value1,value2 in i:
      if key2==1:
        print(image)

1
D:\PTIT\CSDLDPT\Multimedia-Database-System\Mix\Kiwi (02).jpg
D:\PTIT\CSDLDPT\Multimedia-Database-System\Mix\Kiwi (03).jpg
D:\PTIT\CSDLDPT\Multimedia-Database-System\Mix\Kiwi (04).jpg
D:\PTIT\CSDLDPT\Multimedia-Database-System\Mix\Kiwi (07).jpg
D:\PTIT\CSDLDPT\Multimedia-Database-System\Mix\Kiwi (08).jpg
D:\PTIT\CSDLDPT\Multimedia-Database-System\Mix\Kiwi (14).jpg
D:\PTIT\CSDLDPT\Multimedia-Database-System\Mix\Kiwi (15).jpg
D:\PTIT\CSDLDPT\Multimedia-Database-System\Mix\Pear (07).jpg
D:\PTIT\CSDLDPT\Multimedia-Database-System\Mix\Pear (11).jpg
D:\PTIT\CSDLDPT\Multimedia-Database-System\Mix\Pear (12).jpg

D:\PTIT\CSDLDPT\Multimedia-Database-System\Mix\Banana (14).jpg
D:\PTIT\CSDLDPT\Multimedia-Database-System\Mix\Kiwi (01).jpg
D:\PTIT\CSDLDPT\Multimedia-Database-System\Mix\Kiwi (05).jpg
D:\PTIT\CSDLDPT\Multimedia-Database-System\Mix\Kiwi (06).jpg
D:\PTIT\CSDLDPT\Multimedia-Database-System\Mix\Kiwi (09).jpg
D:\PTIT\CSDLDPT\Multimedia-Database-System\Mix\Kiwi (10).jpg
D:\PTIT\CSDLDPT\Mul